In [1]:
import pandas as pd
import numpy as np

In [2]:
print(pd.__version__)
print(np.__version__)

1.4.3
1.21.2


##### points to notice



- categories of joins
- one-to-one joins
- many-to-one joins
- many-to-many joins
- specification of merge key
- the on keyword
- the left_on and right_on keywords
- the left_index and right_index keywords
- specifying set arithmetic joins
- overlapping column name-set suffixes


One essential feature offered by Pandas is its high-performance, in-memory join and
merge operations. If you have ever worked with databases, you should be familiar
with this type of data interaction. The main interface for this is the pd.merge func‐
tion, and we’ll see a few examples of how this can work in practice.

##### Relational Algebra

The behavior implemented in pd.merge() is a subset of what is known as relational
algebra, which is a formal set of rules for manipulating relational data, and forms the
conceptual foundation of operations available in most databases. The strength of the relational algebra approach is that it proposes several primitive operations, which
become the building blocks of more complicated operations on any dataset. With this
lexicon of fundamental operations implemented efficiently in a database or other pro‐
gram, a wide range of fairly complicated composite operations can be performed. <br>

Pandas implements several of these fundamental building blocks in the pd.merge()
function and the related join() method of Series and DataFrames. As we will see,
these let you efficiently link data from different sources.

##### Categories of Joins

The pd.merge() function implements a number of types of joins: the one-to-one,
many-to-one, and many-to-many joins. All three types of joins are accessed via an
identical call to the pd.merge() interface; the type of join performed depends on the
form of the input data. Here we will show simple examples of the three types of
merges, and discuss detailed options further below.

###### One-to-one joins

Perhaps the simplest type of merge expression is the one-to-one join

In [4]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
print(df1); print(df2)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014


To combine this information into a single DataFrame, we can use the pd.merge()
function:

In [5]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


The pd.merge() function recognizes that each DataFrame has an “employee” column,
and automatically joins using this column as a key. The result of the merge is a new
DataFrame that combines the information from the two inputs. Notice that the order
of entries in each column is not necessarily maintained: in this case, the order of the
“employee” column differs between df1 and df2, and the pd.merge() function cor‐
rectly accounts for this. Additionally, keep in mind that the merge in general discards
the index, except in the special case of merges by index (see “The left_index and
right_index

###### Many-to-one joins

Many-to-one joins are joins in which one of the two key columns contains duplicate
entries. For the many-to-one case, the resulting DataFrame will preserve those dupli‐
cate entries as appropriate. Consider the following example of a many-to-one join:

In [6]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                           'supervisor': ['Carly', 'Guido', 'Steve']})
print(df3); print(df4); print(pd.merge(df3, df4))

  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve


The resulting DataFrame has an additional column with the “supervisor” information,
where the information is repeated in one or more locations as required by the inputs.

##### Many-to-many joins

Many-to-many joins are a bit confusing conceptually, but are nevertheless well
defined. If the key column in both the left and right array contains duplicates, then
the result is a many-to-many merge. This will be perhaps most clear with a concrete
example. Consider the following, where we have a DataFrame showing one or more
skills associated with a particular group.

By performing a many-to-many join, we can recover the skills associated with any
individual person:

In [7]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                                     'Engineering', 'Engineering', 'HR', 'HR'], 'skills': ['math', 'spreadsheets', 'coding', 'linux',
                                      'spreadsheets', 'organization']})
print(df1); print(df5); print(pd.merge(df1, df5))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR  organization
  employee        group        skills
0      Bob   Accounting          math
1      Bob   Accounting  spreadsheets
2     Jake  Engineering        coding
3     Jake  Engineering         linux
4     Lisa  Engineering        coding
5     Lisa  Engineering         linux
6      Sue           HR  spreadsheets
7      Sue           HR  organization


These three types of joins can be used with other Pandas tools to implement a wide
array of functionality. But in practice, datasets are rarely as clean as the one we’re
working with here. In the following section, we’ll consider some of the options pro‐
vided by pd.merge() that enable you to tune how the join operations work.

###### Specification of the Merge Key

We’ve already seen the default behavior of pd.merge(): it looks for one or more
matching column names between the two inputs, and uses this as the key. However,
often the column names will not match so nicely, and pd.merge() provides a variety
of options for handling this.

##### The on keyword

Most simply, you can explicitly specify the name of the key column using the on key‐
word, which takes a column name or a list of column names:

In [8]:
print(df1); print(df2); print(pd.merge(df1, df2, on='employee'))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014


This option works only if both the left and right DataFrames have the specified col‐
umn name.

##### The left_on and right_on keywords

At times you may wish to merge two datasets with different column names; for exam‐
ple, we may have a dataset in which the employee name is labeled as “name” rather
than “employee”. In this case, we can use the left_on and right_on keywords to
specify the two column names:

In [9]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
print(df1); print(df3);
print(pd.merge(df1, df3, left_on="employee", right_on="name"))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000


The result has a redundant column that we can drop if desired—for example, by
using the drop() method of DataFrames:

In [10]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


##### The left_index and right_index keywords

Sometimes, rather than merging on a column, you would instead like to merge on an
index. For example, your data might look like this:

In [11]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
print(df1a); print(df2a)

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014


You can use the index as the key for merging by specifying the left_index and/or
right_index flags in pd.merge():

In [12]:
print(df1a); print(df2a);
print(pd.merge(df1a, df2a, left_index=True, right_index=True))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014
                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


For convenience, DataFrames implement the join() method, which performs a
merge that defaults to joining on indices:

In [13]:
print(df1a); print(df2a); print(df1a.join(df2a))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014
                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


##### Specifying Set Arithmetic for Joins

In all the preceding examples we have glossed over one important consideration in
performing a join: the type of set arithmetic used in the join. This comes up when a
value appears in one key column but not the other. Consider this example:

In [14]:
 df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                            'food': ['fish', 'beans', 'bread']},
                           columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                            'drink': ['wine', 'beer']},
                           columns=['name', 'drink'])
print(df6); print(df7); print(pd.merge(df6, df7))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
   name   food drink
0  Mary  bread  wine


Here we have merged two datasets that have only a single “name” entry in common:
Mary. By default, the result contains the intersection of the two sets of inputs; this is
what is known as an inner join. We can specify this explicitly using the how keyword,
which defaults to 'inner':

In [15]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


Other options for the how keyword are 'outer', 'left', and 'right'. An outer join
returns a join over the union of the input columns, and fills in all missing values with
NAs:

In [16]:
print(df6); print(df7); print(pd.merge(df6, df7, how='outer'))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer


##### Overlapping Column Names: The suffixes Keyword

Finally, you may end up in a case where your two input DataFrames have conflicting
column names. Consider this example:

In [17]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                            'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                            'rank': [3, 1, 4, 2]})
print(df8); print(df9); print(pd.merge(df8, df9, on="name"))

   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


Because the output would have two conflicting column names, the merge function
automatically appends a suffix _x or _y to make the output columns unique. If these
defaults are inappropriate, it is possible to specify a custom suffix using the suffixes
keyword:

In [18]:
print(df8); print(df9);
print(pd.merge(df8, df9, on="name", suffixes=["_L", "_R"]))

   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


These suffixes work in any of the possible join patterns, and work also if there are
multiple overlapping columns.